# preprocess wikipedia corpus

In [1]:
pwd

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## load word piece model for tokeniztion usage

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 10.0 MB/s 
     |████████████████████████████████| 7.6 MB 59.2 MB/s 
     |████████████████████████████████| 163 kB 44.8 MB/s 


In [4]:
from transformers import BertTokenizer, BertModel
import numpy as np

## load corpus

In [5]:
class utils(object):
  def __init__(self,path=None):
    self.path=path
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

  def preprocess_wiki(self, path):
    f = open(self.path)
    res = f.read().splitlines()
    f.close()

    vocab=set()
    length=[]
    sentence_arr=[]
    for i in range(len(res)):
      text=res[i]
      tokenized_text = self.tokenizer.tokenize(text)
      # remove sentence which is too short, too long
      if 5<= len(tokenized_text)<=100:
        sentence_arr.append(tokenized_text)
        length.append(len(tokenized_text))
        vocab.update(tokenized_text)

    v2i={v: i for i, v in enumerate(sorted(vocab), start=1)}
    v2i['<PAD>']=0
    v2i["<SEP>"] = len(v2i) # <GO> as start of sequence ,<SEP> as end of sequence
    v2i["<GO>"] = len(v2i) # the total number of tokens should include these special tokens: len(v2i)

    i2v = {i: v for v, i in v2i.items()}  
    return sentence_arr, v2i, i2v, max(length)

  def token_to_idx(self,sentence_arr, v2i):
    sentence_idx=[]
    for i in range(len(sentence_arr)):
      sentence_idx.append([v2i['<GO>']]+[v2i[item] for item in sentence_arr[i]]+[v2i['<SEP>']])
    return sentence_idx

  # add a pad_zero function to align the sentences of various length
  def pad_zero(self, seqs, max_len):
      PAD_ID = 0
      padded = np.full((len(seqs), max_len), fill_value=PAD_ID, dtype=np.int32)
      for i, seq in enumerate(seqs):
          padded[i, :len(seq)] = seq
      return padded

  def get_idx_sentence(self):
    sentence_arr, v2i, i2v, max_len= self.preprocess_wiki(self.path) #input is part of wiki data, for demo usage
    sentence_idx = self.token_to_idx(sentence_arr, v2i)
    # define idx for padding
    PAD_ID= v2i['<PAD>']
    # there is <GO> and <SEP> at start and ending of sentence, so the full length should be 100+2=102
    sentence_idx_padded = self.pad_zero(sentence_idx,max_len+2)

    return sentence_idx_padded.tolist(), v2i



# define the module and gpt class

In [6]:
from torch import Tensor
import torch.nn.functional as f

In [7]:
import torch
from torch import nn

class AttentionHead(nn.Module):
    def __init__(self, dim_in: int, dim_k: int, dim_v: int):
        super().__init__()
        self.q = nn.Linear(dim_in, dim_k)
        self.k = nn.Linear(dim_in, dim_k)
        self.v = nn.Linear(dim_in, dim_v)
        #self.embedding=nn.Embedding() ##

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask: Tensor) -> Tensor:  ## 传入mask 
        query= self.q(query)
        key= self.k(key)
        value=self.v(value)

        temp = query.bmm(key.transpose(1, 2))
        scale = query.size(-1) ** 0.5

        score=temp/scale
        score=score+mask

        softmax = f.softmax(score, dim=-1)
        return softmax.bmm(value)

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads: int, dim_in: int, dim_k: int, dim_v: int):
        super().__init__()
        self.heads = nn.ModuleList(
            [AttentionHead(dim_in, dim_k, dim_v) for _ in range(num_heads)]
        )
        self.linear = nn.Linear(num_heads * dim_v, dim_in)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask) -> Tensor: ## 传入mask
        return self.linear(
            torch.cat([h(query, key, value,mask) for h in self.heads], dim=-1)
        )

In [10]:
def position_encoding(
    seq_len: int, dim_model: int, device: torch.device = torch.device("cpu"),
) -> Tensor:
    pos = torch.arange(seq_len, dtype=torch.float, device=device).reshape(1, -1, 1)
    dim = torch.arange(dim_model, dtype=torch.float, device=device).reshape(1, 1, -1)
    phase = pos / 1e4 ** (dim // dim_model)

    return torch.where(dim.long() % 2 == 0, torch.sin(phase), torch.cos(phase))

In [11]:
def feed_forward(dim_input: int = 512, dim_feedforward: int = 2048) -> nn.Module:
    return nn.Sequential(
        nn.Linear(dim_input, dim_feedforward),
        nn.ReLU(),
        nn.Linear(dim_feedforward, dim_input),
    )

In [12]:
class Residual(nn.Module):
    def __init__(self, sublayer: nn.Module, dimension: int, dropout: float = 0.1): ##不是在这里传入mask
        super().__init__()
        self.sublayer = sublayer
        self.norm = nn.LayerNorm(dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, tensors0: Tensor, tensors1: Tensor, tensors2: Tensor, mask: Tensor) -> Tensor:
        # Assume that the "value" tensor is given last, so we can compute the
        # residual.  This matches the signature of 'MultiHeadAttention'.
        # self.mask=mask
        return self.norm(tensors0 + self.dropout(self.sublayer(tensors0, tensors1, tensors2,mask))) ## 传入mask

In [13]:
## feed fowward network,Residual需要传入mask，这里不用，所以要分别开
class Residual_ffn(nn.Module):
    def __init__(self, sublayer: nn.Module, dimension: int, dropout: float = 0.1): ##不传入mask
        super().__init__()
        self.sublayer = sublayer
        self.norm = nn.LayerNorm(dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, tensors: Tensor) -> Tensor:
        # Assume that the "value" tensor is given last, so we can compute the
        # residual.  This matches the signature of 'MultiHeadAttention'.
        #self.mask=mask ##
        return self.norm(tensors + self.dropout(self.sublayer(tensors)))

In [14]:
class TransformerEncoderLayer(nn.Module):
    def __init__(
        self, 
        dim_model: int = 512, 
        num_heads: int = 6, 
        dim_feedforward: int = 2048, 
        dropout: float = 0.1, 
    ):
        super().__init__()
        dim_k = dim_v = dim_model // num_heads
        self.attention = Residual(
            MultiHeadAttention(num_heads, dim_model, dim_k, dim_v), ## 传入mask
            dimension=dim_model,
            dropout=dropout, 
        )
        self.feed_forward = Residual_ffn(
            feed_forward(dim_model, dim_feedforward),
            dimension=dim_model,
            dropout=dropout,
        )

    def forward(self, src: Tensor,mask: Tensor) -> Tensor: ##传入mask
        src = self.attention(src, src, src,mask) ###传入mask
        #return src 
        return self.feed_forward(src)

In [15]:
class TransformerEncoder(nn.Module):
    def __init__(
        self, 
        num_layers: int = 6,
        dim_model: int = 512, 
        num_heads: int = 8, 
        dim_feedforward: int = 2048, 
        dropout: float = 0.1,
        device: str = 'cpu' 
    ):
        super().__init__()
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(dim_model, num_heads, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])
        self.device= device

    def forward(self, src: Tensor, mask: Tensor) -> Tensor: ##可以传入mask
        seq_len, dimension = src.size(1), src.size(2)
        pos=position_encoding(seq_len,dimension, torch.device(self.device) ) #
        #pos=pos.cuda(0) # load data to gpu
        src += pos
        for layer in self.layers:
            src = layer(src,mask)  ##传入mask

        return src

In [16]:
# start a trial of gpt model testing
class GPT(nn.Module):
    def __init__(
        self, 
        num_encoder_layers: int = 4,
        dim_model: int = 512, 
        num_heads: int = 8, 
        dim_feedforward: int = 2048//2, 
        dropout: float = 0.1, 
        activation: nn.Module = nn.ReLU(),
        n_vocab: int=4,
        device: str = 'cpu'
    ):
        super().__init__()

        self.embedding = nn.Embedding(n_vocab, dim_model)
        
        self.encoder = TransformerEncoder(
            num_layers=num_encoder_layers,
            dim_model=dim_model,
            num_heads=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            device= device
        )

        self.out = nn.Linear(dim_model, n_vocab)



    def forward(self, src: Tensor, mask: Tensor) -> Tensor: ##传入mask
        emb=self.embedding(src)
        enc=self.encoder(emb,mask) ##传入mask
        out=self.out(enc)
        
        return out ##no need softmax, nn.cross_entropy take care of it

# load data

# train model

In [17]:
# Generates a square matrix where the each row allows one word more to be seen

def generate_masks(src):
    seq_len= src.size(1)

    pad_int= [int(seq_len-i) for i in src.count_nonzero(dim=1)]

    mask = torch.tril(torch.ones(seq_len, seq_len) == 1) # Lower triangular matrix
    mask = mask.float()
    mask = mask.masked_fill(mask == 0, -1e9) # Convert zeros to -1e9
    mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0

    mask_arr=[]
    for i in pad_int:
      mask[:,-(i):]= -1e9
      mask_arr.append(mask)

    masks=torch.cat(tuple(mask_arr),dim=0)
    masks=masks.reshape(src.size(0),seq_len,seq_len)
    
    return masks

In [18]:
import time

######
def loss_masked(output, src, loss_fn):
    nonpad_int=src.count_nonzero(dim=1)
    # discard pad elements
    res=[]
    for k,item in enumerate(nonpad_int):
        res.append(src[k][:int(item)])

    loss_res=0
    for i in range(src.size(0)):
        loss_res+=loss_fn(output[i], src[i])

    return loss_res/src.size(0)
######

def train(model, data, batch_size):
    torch.manual_seed(0)
    model.to(device)

    # define loss function (criterion) and optimizer
    loss_fn = nn.CrossEntropyLoss().to(device)  ## this is for classification
    opt = torch.optim.SGD(model.parameters(), 1e-4)


    n=len(data)// batch_size

    # prepare for next word prediction
    t0=time.time()

    for i in range(n):
        src=data[batch_size*i:batch_size*(i+1)]
        src=torch.tensor(src).long()

        _seq=src[:,:-1]
        seq_=src[:,1:]
        masks=generate_masks(_seq)

        
        #put to gpu
        _seq=_seq.to(device)
        seq_=seq_.to(device)
        masks=masks.to(device)

        # Forward pass
        outputs = model(_seq,masks)
        loss=loss_masked(outputs,seq_, loss_fn) # next word prediction

        # the part of padding loss should be removed before backprop
        opt.zero_grad()
        loss.backward()
        opt.step()

        print(loss.detach().item())
    print(time.time()-t0)
        
        #if i%10000==0: np.savetxt('./torch_save_model/gpt_loss_%d.csv'%(i), np.array([loss.detach().item()]))

    # save model parameters after finish training model
    torch.save(model, "./model.pkl")

In [19]:
device

NameError: ignored

In [ ]:
# the entry to start training the model, with data, with specified parameter
if __name__ == '__main__':
    # load cpu or gpu name
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    # model parameter
    MODEL_DIM = 256
    N_LAYER = 4
    N_HEAD = 8
    # training argument
    batch_size = 128
    # load data
    ut = utils(path='/content/drive/MyDrive/goodnlp-github-data/single_gpu_pytorch_gpt/file00')
    d, v2i=ut.get_idx_sentence()
    n_vocab= len(v2i)
    m= GPT(num_encoder_layers= N_LAYER,dim_model= MODEL_DIM, num_heads= N_HEAD, n_vocab=n_vocab, device=device)
    # start training
    train(m, d, batch_size)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  


9.704425811767578
9.671913146972656
9.671069145202637
9.373780250549316
9.636103630065918
9.584640502929688
9.521275520324707
9.513204574584961
9.534192085266113
9.463265419006348
9.231632232666016
9.511894226074219
9.383188247680664
9.34734058380127
9.346139907836914
9.258397102355957
9.286152839660645
9.271854400634766
9.25654125213623
9.264501571655273
